In [1]:
import Modules.model.Segment as Segment
import torch
import Modules.model.Detector as Detector
import Modules.model.Classifier as Classifier
import os
from torchvision.transforms._presets import ObjectDetection
from functools import partial
import numpy as np
import os
import PIL.Image as Image
from Modules.dataloader import base_dataset, DataModule

c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


In [2]:
class PennFudanDataset(base_dataset):

    def __init__(self, transform=None,
                 data_path='../../data/PennFudanPed/'):
        
        
        imgs = list(map(lambda img: os.path.join(data_path, "PNGImages", img), 
                        sorted(os.listdir(os.path.join(data_path, "PNGImages")))))
        masks = list(map(lambda mask: os.path.join(data_path, "PedMasks", mask), 
                        sorted(os.listdir(os.path.join(data_path, "PedMasks")))))
        
        if(transform is None):
            transform = partial(ObjectDetection)()

        super(PennFudanDataset, self).__init__(data=imgs, targets=masks, transform=transform)

    def __parse_mask__(self, mask):
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
        
        return boxes, masks, obj_ids

    def __load_sample__(self, index) -> dict:
        
        img = Image.open(self.data[index]).convert("RGB")
        mask = Image.open(self.targets[index]).convert('L')
        boxes, masks, obj_ids = self.__parse_mask__(mask)

        return {'data': img,
                'target': {'boxes': boxes,
                           'masks': masks,
                           'obj_ids': obj_ids},
                'original': img}
    
    def __transform__(self, sample: dict) -> dict:
        
        boxes = sample['target']['boxes']
        masks = sample['target']['masks']
        num_objs = len(sample['target']['obj_ids'])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        # image_id = torch.tensor([index])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = torch.tensor([0])
        target["area"] = area
        target["iscrowd"] = iscrowd

        sample.update({'target': target})

        return super().__transform__(sample)

In [3]:
dataset = PennFudanDataset(data_path='data\PennFudanPed', 
                           transform=None)

In [5]:
from Modules.dataloader import collate_fn_dict

data_loader = DataModule(dataset, 
                        batch_size=2,
                        num_workers=0,
                        collate_fn=collate_fn_dict)

data_loader.setup(stage = "fit")

for batch in data_loader.train_dataloader():
    print(batch)
    break


{'data': [tensor([[[0.1294, 0.1373, 0.1216,  ..., 0.4314, 0.4314, 0.4196],
         [0.1412, 0.1608, 0.1608,  ..., 0.3608, 0.3804, 0.3725],
         [0.1373, 0.1569, 0.1569,  ..., 0.3020, 0.3137, 0.3216],
         ...,
         [0.7882, 0.7843, 0.7804,  ..., 0.7137, 0.7137, 0.7176],
         [0.7922, 0.7804, 0.7765,  ..., 0.7176, 0.7176, 0.7176],
         [0.7961, 0.7843, 0.7725,  ..., 0.7098, 0.7098, 0.7098]],

        [[0.1647, 0.1725, 0.1569,  ..., 0.3725, 0.3725, 0.3608],
         [0.1765, 0.1961, 0.1961,  ..., 0.3137, 0.3216, 0.3255],
         [0.1725, 0.1922, 0.1922,  ..., 0.2588, 0.2667, 0.2784],
         ...,
         [0.7843, 0.7804, 0.7765,  ..., 0.6980, 0.6980, 0.7020],
         [0.7882, 0.7765, 0.7725,  ..., 0.7020, 0.7020, 0.7020],
         [0.7922, 0.7804, 0.7686,  ..., 0.6941, 0.6941, 0.6941]],

        [[0.1451, 0.1529, 0.1373,  ..., 0.3843, 0.3843, 0.3725],
         [0.1647, 0.1765, 0.1843,  ..., 0.3216, 0.3333, 0.3333],
         [0.1686, 0.1804, 0.1882,  ..., 0.2745, 

In [6]:
import torch.optim as optim
from typing import List

def get_optimizer(parameters: List, settings: dict) -> optim.Optimizer:
    """
    Set up learning optimizer
    Args:
        parameters: model's parameters
        settings: settings hyperparameters
    Returns:
        optimizer: optimizer
    """
    optimizer_mapping = {
        'adam': lambda: optim.Adam(parameters, lr=settings['lr'], weight_decay=settings['weight_decay']),
        'sgd': lambda: optim.SGD(parameters, lr=settings['lr'], weight_decay=settings['weight_decay'], momentum=settings['momentum']),
    }

    optimizer_creator = optimizer_mapping.get(settings['optimizer'])

    if optimizer_creator is not None:
        return optimizer_creator()
    else:
        raise NotImplementedError()

# Example usage:
model_parameters = [torch.nn.Parameter(torch.randn(20, 20, requires_grad=True))]
optimizer_settings = {'optimizer': 'adam', 'lr': 0.001, 'weight_decay': 0.0001}
optimizer_instance = get_optimizer(model_parameters, optimizer_settings)

In [9]:
metric_modes = {'acc': 'max', 'dice': 'max', 'mAP': 'max'}
mode = metric_modes.get('acc', 'min')
print(mode)

max


In [1]:
from Modules.ultis import get_basic_callbacks

callbacks = get_basic_callbacks(metric_name='acc',
                                ckpt_path='checkpoints',
                                early_stopping=True)


c:\Users\197796\Anaconda3\envs\AISeed\lib\site-packages\neptune\common\warnings.py:62: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  warnings.warn(


In [ ]:
from Modules.ultis import get_trainer

trainer = get_trainer(logger=True,
                      gpu_ids=[0],
                      n_gpu=1,
                      metric_name='acc',
                      ckpt_path='checkpoints',
                      early_stopping=True)
